In [15]:
#import pandas as pd
import re
import ipdb
import numpy as np
from multiprocessing import Process, Queue, cpu_count
import queue

In [16]:
#!cat taxi_log_2008_by_id/*.txt > total.txt

In [17]:
def distance(a,b):
    return(np.sum((a-b)**2))

In [18]:
districts = {"Dongcheng":np.array([39.935539,116.405640]),
             "Xicheng": np.array([39.813818,116.363625]), 
             "Chongwen": np.array([39.896949, 116.414961]), 
             "Xuanwu": np.array([39.89426, 116.382425]), 
             "Chaoyang": np.array([39.922478,116.443710]),
             "Fengtai": np.array([39.858425,116.287148]), 
             "Shijingshan":np.array([39.919441,116.168488]),
             "Haidian": np.array([39.992271, 116.277328]), 
             "Mentougou": np.array([39.938998,116.094757]), 
             "Fangshan": np.array([39.698,115.993]), 
             "Tongzhou": np.array([39.902392,116.651314]), 
             "Shunyi":np.array([40.127577,116.655291]), 
             "Changping": np.array([40.21612,116.23471]),
             "Daxing": np.array([39.74025,116.32693]), 
             "Pinggu": np.array([40.142351,117.100989]), 
             "Huairou":np.array([40.31333, 116.62888]), 
             "Miyun":np.array([40.376299, 116.842993]), 
             "Yanqing":np.array([40.526731,116.189010])}

In [19]:
def best_district(coordinate):
    best = np.infty
    for district, coord in districts.items():
        dist_tmp = distance(coord,
                            coordinate)
        if dist_tmp < best:
            best      = dist_tmp
            distretto = district
    return distretto

In [21]:
f = open('total.txt','r')

In [22]:
%%time
lines = f.readlines()

CPU times: user 1.98 s, sys: 952 ms, total: 2.93 s
Wall time: 2.93 s


In [23]:
cores = cpu_count()
split = int(len(lines)/8)

In [24]:
%%time
splitted=[lines[:split]]
for i in range(1,cores-1):
    splitted.append(lines[i*split:(i+1)*split])
splitted.append(lines[(cores-1)*split:])

CPU times: user 1.15 s, sys: 284 ms, total: 1.43 s
Wall time: 1.42 s


In [25]:
del(lines) # reduce ram stress

In [26]:
def add_district(split,index):
    file_name = "total_district_" + str(index+1) + ".txt"
    fw = open(file_name,'w')
    for i in range(len(split)):
        split[i]    = split[i].strip()
        obj         = split[i].split(',')
        coordinates = np.array([float(obj[3]),float(obj[2])])
        location    = best_district(coordinates)
        fw.write(split[i] + "," + location + '\n')

In [27]:
len(splitted[6])

2207873

In [ ]:
%%time
threads = []
#result = Queue()
for i in range(len(splitted)):
    process = Process(target=add_district,args=[splitted[i],i])
    process.start()
    threads.append(process)
for process in threads:
    process.join()

In [30]:
!cat total_district_ > total_district.txt 

In [7]:
#%%time
#with open("total.txt") as f:
#    for line in f:
#        line        = line.replace('\n','')
#        obj         = line.split(',')
#        coordinates = np.array([float(obj[3]),float(obj[2])])
#        location    = best_district(coordinates)
#        line        = line + "," +location
#        fw.write(line+'\n')
#fw.close()

CPU times: user 29min 54s, sys: 3.28 s, total: 29min 58s
Wall time: 29min 58s
